In [52]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import warnings
warnings.filterwarnings('ignore')

In [53]:
### Use GPU 
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))
gpu_devices = tf.config.list_physical_devices('GPU')
gpu_devices[0][0]


'/physical_device:GPU:0'

In [54]:
### Loading MNIST dataset 
from keras.datasets import mnist

In [55]:
### Unzipping MNIST dataset
(xTrain,yTrainLabel),(xTest,yTestLabel) = mnist.load_data()

In [56]:
### One-hot enocding target labels
yTrainCat = tf.keras.utils.to_categorical(yTrainLabel, num_classes = 10, dtype = 'float32')
yTestCat = tf.keras.utils.to_categorical(yTestLabel, num_classes = 10, dtype = 'float32')


In [57]:
### Training and testing data
xTrain = (xTrain/255).astype('float32')
xTest = (xTest/255).astype('float32')

In [58]:
### Model ### --- layer adjustments

#creating lists for column names 
model_names = []
first_layer_nodes = []
second_layer_nodes = []
first_layer_activation = []
second_layer_activation = []
output_layer_activation = []
training_loss = []
training_accuracy = []
validation_loss = []
validation_accuracy = []
batch_size = [] 
num_of_epochs = []
test_loss = []
test_accuracy = []
optimizer_function = []
loss_function = []

def model_builder(training_dataset = xTrain, 
                  training_labels = yTrainCat, 
                  testing_dataset = xTest, 
                  testing_labels = yTestCat):
  
  ### ----- Local Variables ----- ###
  counter = 0
  firstActive = 'relu'
  secondActive = 'relu'
  outputActive = 'sigmoid'
  firstLayerNodes = [100,150,200] # number of nodes in first layer list
  secondLayerNodes = [50,87,125]  # number of nodes in second layer list

  ### ----- Creating the full-dense network ----- ###

  for i in firstLayerNodes: # will iterate through first layer nodes
    for j in secondLayerNodes: # iterate through second layer nodes
      counter += 1
      inputLayer = tf.keras.Input(shape = (28,28), name = 'input_layer')
      flattenLayer = tf.keras.layers.Flatten(name = 'flatten_layer')(inputLayer)
      denseLayer1 = tf.keras.layers.Dense(units = i, activation = firstActive, name = 'dense_layer_1')(flattenLayer)
      denseLayer2 = tf.keras.layers.Dense(units = j, activation = secondActive, name = 'dense_layer_2')(denseLayer1)
      outputLayer = tf.keras.layers.Dense(units = 10, activation = outputActive, name = 'output_layer')(denseLayer2)

      # appending all hyperparameters into lists
      first_layer_nodes.append(i) # append number of first layer nodes 
      second_layer_nodes.append(j) # append number of second layer nodes
      first_layer_activation.append(firstActive) # append first layer activation function
      second_layer_activation.append(secondActive) # append second layer activation function
      output_layer_activation.append(outputActive) # append output layer activation function

      modelName = f'NN_Model{str(counter)}' # generating model names to put into list
      model_names.append(modelName) # appending name to model_names list

      model = tf.keras.Model(inputs = inputLayer, outputs = outputLayer, name = modelName)

      ### ----- Model Parameters ----- ###

      optFunction = tf.keras.optimizers.Adam(learning_rate =  0.025) # optimizer function
      lossFunction = tf.keras.losses.BinaryCrossentropy() # loss function

      optimizer_function.append(str(optFunction)) # append optimizer function
      loss_function.append(str(lossFunction)) # append loss function

      ### ----- Compiler ----- ###
      model.compile(
        optimizer = optFunction,
        loss = lossFunction,
        metrics = tf.keras.metrics.Accuracy()
      )

      ##### ----- Fitting the model ----- ###
      print(f'first layer: {i}, second layer {j}') 

      tf.random.set_seed(42)

      bSize = 200
      epoch = 250 # change back to 250
      vSplit = 0.1
      trainModel = model.fit(
          x = xTrain,
          y = yTrainCat,
          batch_size = bSize,
          epochs = epoch,
          validation_split = vSplit
      )

      batch_size.append(bSize) # append batch size
      num_of_epochs.append(epoch) # append number of epochs 
      
      ### ----- Results ----- ###
      # appending the validation accuracy into the list
      training_loss.append(min(trainModel.history['loss'])) # append training loss
      training_accuracy.append(max(trainModel.history['accuracy'])) # append training accuracy
      validation_loss.append(min(trainModel.history['val_loss'])) # append validation loss
      validation_accuracy.append(max(trainModel.history['val_accuracy'])) # append validation accuracy 
                                 
      # appending the test accuracy into the list
      finalResults = model.evaluate(xTest,yTestCat)

      test_loss.append(finalResults[0]) # append test loss
      test_accuracy.append(finalResults[1]) # append test accuracy

In [59]:
with tf.device('GPU:0'):
    model_builder()


first layer: 100, second layer 50
Epoch 1/250
270/270 [==============================] - 2s 7ms/step - loss: 0.0655 - accuracy: 4.4444e-05 - val_loss: 0.0289 - val_accuracy: 0.0000e+00
Epoch 2/250
270/270 [==============================] - 2s 6ms/step - loss: 0.0301 - accuracy: 1.9630e-04 - val_loss: 0.0225 - val_accuracy: 6.5000e-04
Epoch 3/250
270/270 [==============================] - 1s 5ms/step - loss: 0.0236 - accuracy: 6.0000e-04 - val_loss: 0.0206 - val_accuracy: 5.1667e-04
Epoch 4/250
270/270 [==============================] - 1s 5ms/step - loss: 0.0210 - accuracy: 0.0013 - val_loss: 0.0241 - val_accuracy: 0.0025
Epoch 5/250
270/270 [==============================] - 1s 5ms/step - loss: 0.0192 - accuracy: 0.0034 - val_loss: 0.0230 - val_accuracy: 0.0032
Epoch 6/250
270/270 [==============================] - 1s 5ms/step - loss: 0.0178 - accuracy: 0.0066 - val_loss: 0.0192 - val_accuracy: 0.0053
Epoch 7/250
270/270 [==============================] - 1s 5ms/step - loss: 0.0166 - 

In [60]:
model_data = {
    'model_names' : model_names,
    'first_layer_nodes' : first_layer_nodes,
    'first_layer_activation' : first_layer_activation,
    'second_layer_nodes' : second_layer_nodes,
    'second_layer_activation' : second_layer_activation,
    'output_layer_activation' : output_layer_activation,
    'optimizer_function' : optimizer_function,
    'loss_function' : loss_function,
    'batch_size' : batch_size,
    'num_of_epochs' : num_of_epochs,
    'training_loss' : training_loss,
    'training_accuracy' : training_accuracy,
    'validation_loss' : validation_loss,
    'validation_accuracy' : validation_accuracy,
    'test_loss' : test_loss,
    'test_accuracy' : test_accuracy
}

### This model varied the percepton/node layers

In [61]:
df = pd.DataFrame(data = model_data)
df = df.set_index('model_names')

In [62]:
df[['training_loss','training_accuracy','validation_loss','validation_accuracy','test_loss','test_accuracy']] = df[['training_loss','training_accuracy','validation_loss','validation_accuracy','test_loss','test_accuracy']].applymap(lambda x: round(x,4))

In [65]:
max_test_acc = max(df.test_accuracy)

def highlighter(cell_value):
    
    highlight = 'background-color: green'
    default = ''

    if cell_value == max_test_acc:
        return highlight
    else:
        return default
    
df.style.applymap(highlighter)

,first_layer_nodes,first_layer_activation,second_layer_nodes,second_layer_activation,output_layer_activation,optimizer_function,loss_function,batch_size,num_of_epochs,training_loss,training_accuracy,validation_loss,validation_accuracy,test_loss,test_accuracy
model_names,,,,,,,,,,,,,,,
NN_Model1,100,relu,50,relu,sigmoid,,,200,250,0.007100,0.915900,0.019200,0.910600,0.311100,0.896000
NN_Model2,100,relu,87,relu,sigmoid,,,200,250,0.008100,0.915500,0.019700,0.915400,0.309400,0.884000
NN_Model3,100,relu,125,relu,sigmoid,,,200,250,0.011600,0.825500,0.021900,0.816600,0.345000,0.797400
NN_Model4,150,relu,50,relu,sigmoid,,,200,250,0.006900,0.913500,0.019900,0.907100,0.315300,0.861600
NN_Model5,150,relu,87,relu,sigmoid,,,200,250,0.009000,0.922000,0.019600,0.919700,0.268700,0.866800
NN_Model6,150,relu,125,relu,sigmoid,,,200,250,0.009600,0.849700,0.020800,0.846100,0.250000,0.821600
NN_Model7,200,relu,50,relu,sigmoid,,,200,250,0.007600,0.940800,0.018700,0.936400,0.315500,0.916000
NN_Model8,200,relu,87,relu,sigmoid,,,200,250,0.008500,0.903000,0.020200,0.917600,0.342800,0.881700
NN_Model9,200,relu,125,relu,sigmoid,,,200,250,0.007800,0.867300,0.020500,0.872300,0.374800,0.827700
